In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e9/sample_submission.csv
/kaggle/input/playground-series-s4e9/train.csv
/kaggle/input/playground-series-s4e9/test.csv


In [2]:
#%pip install requests
#%pip install tabulate
#%pip install future

# Required for plotting:
#%pip install matplotlib

#!sudo apt update
#!sudo apt install openjdk-11-jdk

In [3]:
!pip install h2o
!pip install requests
!pip install tabulate
!pip install future

In [4]:
%%time
%pip install catboost
#%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
#%pip install h2o
%pip install tqdm
#%pip install wandb
#%pip install sweetviz
#%pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU times: user 1.95 s, sys: 409 ms, total: 2.36 s
Wall time: 1min 25s


In [5]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from pathlib import Path
import os

Load dataset

In [6]:
path = Path('/kaggle/input/playground-series-s4e9/')
path

PosixPath('/kaggle/input/playground-series-s4e9')

In [7]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [8]:
#x = train_df.columns
#y = "price"
#x.remove(y)

In [9]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpynq9c5qj
  JVM stdout: /tmp/tmpynq9c5qj/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpynq9c5qj/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,2 months and 14 days
H2O_cluster_name:,H2O_from_python_unknownUser_1he85y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [10]:
train_data = h2o.H2OFrame(train_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
%%time
aml = H2OAutoML(max_runtime_secs=3600*10,seed=42)
aml.train(y='price', training_frame=train_data)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 6min 47s, sys: 25.8 s, total: 7min 12s
Wall time: 9h 57min 11s


key,value
Stacking strategy,cross_validation
Number of base models (used / total),21/197
# GBM base models (used / total),9/63
# XGBoost base models (used / total),7/117
# DeepLearning base models (used / total),4/14
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [12]:
# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_4_AutoML_1_20240924_170144,72571.7,5.26664e+09,19161.5,0.53103,5.26664e+09,39611,0.427728,StackedEnsemble
StackedEnsemble_AllModels_3_AutoML_1_20240924_170144,72579.7,5.26781e+09,19186.9,0.531639,5.26781e+09,37837,0.388577,StackedEnsemble
StackedEnsemble_AllModels_6_AutoML_1_20240924_170144,72588.6,5.2691e+09,19120.6,0.536786,5.2691e+09,106532,0.536355,StackedEnsemble
StackedEnsemble_Best1000_1_AutoML_1_20240924_170144,72593.9,5.26987e+09,19169,0.531185,5.26987e+09,48091,0.445856,StackedEnsemble
StackedEnsemble_BestOfFamily_4_AutoML_1_20240924_170144,72602.8,5.27116e+09,19290.5,0.536561,5.27116e+09,1970,0.108054,StackedEnsemble
StackedEnsemble_BestOfFamily_6_AutoML_1_20240924_170144,72615.5,5.27301e+09,19232.9,0.539969,5.27301e+09,6555,0.104303,StackedEnsemble
StackedEnsemble_BestOfFamily_3_AutoML_1_20240924_170144,72621.2,5.27384e+09,19289.7,0.536662,5.27384e+09,1903,0.108194,StackedEnsemble
StackedEnsemble_AllModels_2_AutoML_1_20240924_170144,72621.7,5.27392e+09,19289.4,0.536778,5.27392e+09,1944,0.085145,StackedEnsemble
DeepLearning_1_AutoML_1_20240924_170144,72774.1,5.29607e+09,19647.5,nan,5.29607e+09,77228,0.030083,DeepLearning
StackedEnsemble_BestOfFamily_5_AutoML_1_20240924_170144,72776,5.29635e+09,19171.6,nan,5.29635e+09,31370,0.259175,StackedEnsemble


In [13]:
test_data = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
leaderboard = aml.leaderboard
print(leaderboard)
best_model = aml.leader
print(best_model)

model_id                                                    rmse          mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_4_AutoML_1_20240924_170144     72571.7  5.26664e+09  19161.5    0.53103                5.26664e+09
StackedEnsemble_AllModels_3_AutoML_1_20240924_170144     72579.7  5.26781e+09  19186.9    0.531639               5.26781e+09
StackedEnsemble_AllModels_6_AutoML_1_20240924_170144     72588.6  5.2691e+09   19120.6    0.536786               5.2691e+09
StackedEnsemble_Best1000_1_AutoML_1_20240924_170144      72593.9  5.26987e+09  19169      0.531185               5.26987e+09
StackedEnsemble_BestOfFamily_4_AutoML_1_20240924_170144  72602.8  5.27116e+09  19290.5    0.536561               5.27116e+09
StackedEnsemble_BestOfFamily_6_AutoML_1_20240924_170144  72615.5  5.27301e+09  19232.9    0.539969               5.27301e+09
StackedEnsemble_BestOfFamily_3_AutoML_1_20240924_170144  72621.2  5.27384e+09  19289.7    0.536662               5.27384e+09
S

In [15]:
test_data = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
predictions = best_model.predict(test_data)
predictions_df = predictions.as_data_frame()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'model' has levels not trained on: ["allroad 2.7T"]
  warnings.warn(w)
/opt/conda/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'engine' has levels not trained on: ["177.0HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas"]
  warnings.warn(w)
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [17]:
predictions.shape

(125690, 1)

In [18]:
predictions_df

,predict
0,16379.907164
1,79550.352629
2,53113.174976
3,25814.297781
4,29607.985382
...,...
125685,30154.490536
125686,50106.602321
125687,18385.553683
125688,14129.881618


In [19]:
sub_df['price'] =(predictions_df['predict'].values)

In [20]:
sub_df.to_csv('submission.csv', index=False)

In [21]:
!ls

__notebook__.ipynb  submission.csv


In [22]:
#aml = H2OAutoML(max_runtime_secs=3600,seed=42)
#aml = H2OAutoML(max_runtime_secs=900,seed=42)
#aml = H2OAutoML(max_runtime_secs=3600, seed=42, sort_metric="RMSE")
#aml.train(y='price', training_frame=train_data)